<a href="https://colab.research.google.com/github/JozefSL/pyNotes/blob/main/Kalman/DPR_ProdRpLd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Install filterpy and restart runtime
# this command hides the cell output
%%capture
!pip install filterpy
import os
#os.kill(os.getpid(),9)

In [2]:
#@title Import packages from Google and get EIA logo
import numpy as np
import pandas as pd
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
from filterpy.stats import plot_covariance_ellipse
from filterpy.kalman import predict
from filterpy.kalman import update
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import r2_score 
import ipywidgets as widgets

eiaLgFile = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/49/Eia-logomark.svg/640px-Eia-logomark.svg.png"
eiaLogo = [dict(source = eiaLgFile,
                       x=1.06, y=-0.06,
                       sizex=0.15, sizey=0.15,
                       xanchor="center", yanchor="bottom")]


In [3]:
#@title Select DPR Region 
dprRegions = ['Anadarko Region','Bakken Region','Eagle Ford Region','Niobrara Region','Permian Region', 'Marcellus Region','Utica Region','Appalachia Region','Haynesville Region']
dprR = widgets.Dropdown(options=dprRegions, description='DPR_Region:', disabled=False)
dprR

Dropdown(description='DPR_Region:', options=('Anadarko Region', 'Bakken Region', 'Eagle Ford Region', 'Niobrar…

In [4]:
#@title Select oil or gas commodity 
dprCommodity = ['oil','gas']
dprC = widgets.Dropdown(options=dprCommodity, description='DPR_Com:', disabled=False)
dprC

Dropdown(description='DPR_Com:', options=('oil', 'gas'), value='oil')

In [5]:
dprC.value

'oil'

In [6]:
#@title Import rig and production data from the DPR region
file = r"https://www.eia.gov/petroleum/drilling/xls/dpr-data.xlsx"
data = pd.read_excel(file, sheet_name=dprR.value, skiprows=2, ) #usecols=[0,1,4,7]) #index_col=0, , nrows=numRows)
data.columns = ['Month', 'BH','RPo','LDo','PRo','RPg','LDg','PRg']
#data.head(4)
data['BH2MF'] = data['BH'].shift(2)
data['dPRo'] = data['PRo'].diff()
data['dPRg'] = data['PRg'].diff()
data['dBH2MF'] = data['BH2MF'].diff()
data = data.fillna(method="backfill")
data = data.fillna(method="ffill")
data.tail()

,Month,BH,RPo,LDo,PRo,RPg,LDg,PRg,BH2MF,dPRo,dPRg,dBH2MF
192,2023-02-01,353.0,1056.7,-342059.4,5626577.4,2094.6,-619346.4,22185961.5,350.0,-25557.8,182727.1,1.0
193,2023-03-01,349.0,1055.9,-344258.3,5657159.5,2071.6,-624802.0,22296564.3,355.0,30582.1,110602.8,5.0
194,2023-04-01,356.0,1055.0,-349936.9,5679653.1,2050.8,-628230.8,22392282.6,353.0,22493.6,95718.3,-2.0
195,2023-05-01,356.0,1054.2,-355192.9,5692375.9,2030.3,-631243.2,22469627.6,349.0,12722.8,77345.0,-4.0
196,2023-06-01,356.0,1053.4,-359904.4,5707466.3,2010.2,-633740.3,22551532.1,356.0,15090.4,81904.5,7.0


In [71]:
from google.colab import files
uploaded = files.upload() # PermianData.csv

Saving PermianData.csv to PermianData.csv


In [72]:
data = pd.read_csv('PermianData.csv')

In [79]:
data.tail()

,Month,BH,RPg,LDg,PRg,RPo,LDo,PRo,PR,RP,LD
184,7/1/2022,337,1844.293673,-2.180376e+05,21164465.32,947.524265,-234947.5753,5261672.935,5261672.935,947.524265,-234947.5753
185,8/1/2022,344,2933.827832,-7.782604e+05,21395441.71,1195.727307,-343778.9677,5329224.161,5329224.161,1195.727307,-343778.9677
186,9/1/2022,349,2710.704489,-1.914736e+05,21974210.27,1212.902483,-282443.0280,5470084.100,5470084.100,1212.902483,-282443.0280
187,10/1/2022,346,2372.185064,-7.705496e+05,21936986.52,1206.173224,-314402.6643,5506934.161,5506934.161,1206.173224,-314402.6643
188,11/1/2022,343,2633.853061,-1.018800e+06,21737049.20,1190.569193,-377634.8416,5515499.767,5515499.767,1190.569193,-377634.8416


In [398]:
#@title Kalman Filter setup
# Step units in months
dt = 1

if dprC.value == "oil":
    data['PR'] = data['PRo']
    #data['dPR'] = data['dPRo']
    data['RP'] = data['RPo']
    data['LD'] = data['LDo']
else:
    data['PR'] = data['PRg']
    #data['dPR'] = data['dPRg']
    data['RP'] = data['RPg']
    data['LD'] = data['LDg']

orv = data['PR'].iloc[0]
#orr = data['dPR'].iloc[0]
orp = data['RP'].iloc[0]
old = data['LD'].iloc[0]

# x - Origin state estimate vector
x = np.array([orv, orp, old]).T

# Q - Process noise matrix
#Q = Q_discrete_white_noise(dim=3, dt=1, var=.85)
Q = np.array([[0.25, 0.15, 0.1],
              [0.15, 0.11, 0.1],
              [0.1, 0.1, 0.1]])

# P - Covariance matrix
P = np.diag([1, 1, 1])

# R - Measurement noise matrix
R = np.diag([0.2, 0.2, .39])


# H - Measurement function
H = np.array([[1., 0., 0.],
              [0., 1., 0.],
              [0., 0., 1.]])


# F - State transition matrix
F = np.array([[1, 0, 1],
              [0, 1, 0],
              [0, 0, 1]])

# B - Measurement function

B = np.array([x[1], 0, 0])



# Measurements for update 
zsu = data[['PR','RP','LD','BH','BH','BH']].to_numpy()

# System prediction 
Xp = np.empty((0,3), int)

# System update
Xu = np.empty((0,3), int)
for zs  in zsu:
    z = zs[0:3]
    u = zs[3:6]
 
    #u[0]=0
    #u[2]=0
    #print(z)
    #B = np.array([x[1], 0, 0])
    x, P = predict(x=x, P=P, F=F, Q=Q, u=u, B=B, alpha=0.99)    #x, P = predict(x=x, P=P, F=F, Q=Q, u=u)
    Xp = np.vstack([Xp, x])

    x, P = update(x, P, z, R, H,)
    Xu = np.vstack([Xu, x])
   
print('R^2:', r2_score(Xp[:,0], data.PR),'|| lastEstimate:', x, '|| lastRigCount:',u)  

R^2: 0.9787842541568947 || lastEstimate: [5346990.14976943  169936.16784417 -144479.51978569] || lastRigCount: [343. 343. 343.]


In [396]:
#zsu
B

array([nan,  0.,  0.])

In [385]:
u

array([343., 343., 343.])

In [387]:
x[1]

100584.56540825804

In [78]:
x[1]

12310.605792133203

In [109]:
#Xp
#Q
np.array([orv, orp, old]).T

array([ 8.51889516e+05,  5.18184000e+01, -1.82338491e+04])

In [392]:
#@title Plot production data as predicted and reported or estimated in DPR report
fig = go.Figure(px.scatter(title="%s production estimates using Kalman filter" % (dprR.value + " " + dprC.value)))
fig.add_trace(go.Scatter(x=data.Month, y=Xp[:,0], mode='markers',name='PredictPR'))
fig.add_trace(go.Scatter(x=data.Month, y=data.PR, mode='markers',name='Reported'))
fig.add_trace(go.Scatter(x=data.Month, y=Xu[:,0], mode='lines+markers',name='UpdatePR'))
fig.layout.images = eiaLogo
fig.show()

In [358]:
fig = go.Figure(px.scatter(title="%s monthly changes in rig productivity estimates using Kalman filter" % (dprR.value)))
fig.add_trace(go.Scatter(x=data.Month, y=Xp[:,1], mode='markers',name='PredictPR'))
fig.add_trace(go.Scatter(x=data.Month, y=data.RP, mode='markers',name='Reported'))
fig.add_trace(go.Scatter(x=data.Month, y=Xu[:,1], mode='lines+markers',name='UpdatePR'))
fig.layout.images = eiaLogo
fig.show()

In [321]:
fig = go.Figure(px.scatter(title="%s monthly changes in legacy declines estimates using Kalman filter" % (dprR.value)))
fig.add_trace(go.Scatter(x=data.Month, y=Xp[:,2], mode='markers',name='PredictPR'))
fig.add_trace(go.Scatter(x=data.Month, y=data.LD, mode='markers',name='Reported'))
fig.add_trace(go.Scatter(x=data.Month, y=Xu[:,2], mode='lines+markers',name='UpdatePR'))
fig.layout.images = eiaLogo
fig.show()

In [ ]:
dataX = data[['BH2MF','dBH2MF', 'PR']]
dataY = data['dPR']

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()
model.fit(dataX, dataY)
predictions = model.predict(dataX)

In [ ]:
#print(predictions)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['Month'], y=predictions, name='Predictions'))
fig.add_trace(go.Scatter(x=data['Month'], y=data['dPR'], name='Data'))

# Add title and axis labels
fig.update_layout(title='Sample Line Plot', xaxis_title='X-axis', yaxis_title='Y-axis')

# Show the plot
fig.show()